In [58]:
import cv2
import time
import numpy as np

Update the  Hue-Saturation-Value of the colour according to your cloth colour
RGB image has mixed intensity and color information for different color channels 
HSV color has separate information of the color and intensity from each other. This makes HSV color space more robust to lighting changes

OpenCV reads a given image in the BGR format by default

In [59]:
# recording from the webcam
cap_video = cv2.VideoCapture(0)
w = cap_video.get(cv2.CAP_PROP_FRAME_WIDTH);
h = cap_video.get(cv2.CAP_PROP_FRAME_HEIGHT); 

# writing the ouput video object
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
result = cv2.VideoWriter('cloak_output.mp4', fourcc, 20.0, (int(w),int(h)))
    
# set the camera sleep time to 2 & background to 0, before starting
time.sleep(2)
count = 0
background = 0

In [60]:
# To save background image, Capture the background in range of 60

for i in range(60):
    out_val, background = cap_video.read()
    if out_val == False :
        continue

# flip the background frame
background = np.flip(background, axis=1)



Morphology needs two inputs, one is original image, second one is called structuring element or kernel which decides the nature of operation. 
it has 2 basic operators Erosion and Dilation. 

Erosion erodes away the boundaries of foreground object.A pixel in the original image (either 1 or 0) will be considered 1 only if all the pixels under the kernel is 1, otherwise it is eroded (made to zero).
So the thickness or size of the foreground object decreases.It is useful for removing small white noises.

Dilation increases the object area. A pixel element is '1' if atleast one pixel under the kernel is '1'. 

We will do a morphology open ( erosion followed by dilation) and dilation for replacing the covered coloured object portion with a mask image in each frame

In [61]:
# read the background from the webcamera, until camera is open
while (cap_video.isOpened()):
    out_val, pic = cap_video.read()
    if not out_val:
        break
    count += 1
    pic = np.flip(pic, axis=1)
    
    # For more robust to lighting changes, Converting the color space from BGR to HSV
    hsv = cv2.cvtColor(pic, cv2.COLOR_BGR2HSV)


  # Generate masks to detect  color, setting the lower and upper range for mask1 
    lower_pink = np.array([100, 100, 30])        
    upper_pink = np.array([10, 255, 255]) 
    mask1 = cv2.inRange(hsv, lower_pink, upper_pink) 
    # setting the lower and upper range for mask2  
    lower_pink = np.array([125, 100, 30]) 
    upper_pink = np.array([255, 255, 255]) 
    mask2 = cv2.inRange(hsv, lower_pink, upper_pink) 
    
    final_mask = mask1 + mask2
    
    final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations = 2)
    final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8), iterations = 1)
    
        # bitwise_not operation will create an inverted mask to separate out the covered object color from the frame
    inv_mask = cv2.bitwise_not(final_mask)

    # bitwise_and operation with the inverted mask will separate the color part out of the frame
    out_img1 = cv2.bitwise_and(pic, pic, mask=inv_mask)

    # bitwise_and operation will create image showing static background frame only for the masked region
    out_img2 = cv2.bitwise_and(background, background, mask=final_mask)

     # Generating the final output 
    Output = cv2.addWeighted(out_img1, 1, out_img2, 1, 0)
    #Output = cv2.resize(Output, (800, 600))
    
    #fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #result = cv2.VideoWriter('cloak_output.avi', fourcc, 20.0, (int(w),int(h)))
    
    result.write(Output)
    
    cv2.imshow("Invisible Cloak", Output)
    key = cv2.waitKey(25)
    if key == ord('n') or key == ord('p'):
        break

cap_video.release()
result.release()
cv2.destroyAllWindows()
